# Agenda: Cleaning data

1. `NaN` and cleaning it up
    - Series
    - Data frames
    - Two techniques: (a) replacing and (b) removing
2. Nullable types
3. Interpolation
4. Replacement of values

# Why clean our data? Because the real world is messy

- Sensors go dead
- People make mistakes
- People don't report data on time
- Weird errors

We have to balance out cleaning out the bad data, but also not getting rid of too much data. If we're data purists, then we run the risk of not having enough data to work with at all.

# `NaN` -- what is it, and how can we handle it?

Remember that `NaN` stands for "not a number," and it is a float value. It isn't equal to anything, including to itself. 

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
s = Series([10, 20, np.nan, 40, 50])

In [3]:
s

0    10.0
1    20.0
2     NaN
3    40.0
4    50.0
dtype: float64

In [4]:
s.astype(np.int64)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [5]:
# one way to get rid of NaN is to use the .fillna method
# this replaces all NaN values with whatever value we give

s.fillna(5)

0    10.0
1    20.0
2     5.0
3    40.0
4    50.0
dtype: float64

In [6]:
# I can also calculate a value, and insert it there

s.fillna(s.mean())   # this is a common technique

0    10.0
1    20.0
2    30.0
3    40.0
4    50.0
dtype: float64

# Don't use inplace=True!

`fillna` and many other methods in Pandas have an optional keyword argument, `inplace`, where if I say `inplace=True`, then the series/data frame is modified, and we get `None` back from our operation.

This sounds like it'll save memory and be really convenient. It is neither! It doesn't save memory, and it means that we cannot do method chaining, because we get `None` back. The core Pandas developers keep threatening to deprecate and then remove the `inplace=True` option.

In [7]:
s = Series([10, 20, np.nan, 40, 50, 60, 70, np.nan, 90, 100])
s.fillna(s.mean())

0     10.0
1     20.0
2     55.0
3     40.0
4     50.0
5     60.0
6     70.0
7     55.0
8     90.0
9    100.0
dtype: float64

In [8]:
# there is another option, namely dropna
# as you can imagine from its name, it returns a new series without any of the 
# original series' NaN values

s.dropna()

0     10.0
1     20.0
3     40.0
4     50.0
5     60.0
6     70.0
8     90.0
9    100.0
dtype: float64

In [9]:
# you can still handle the indexes via .iloc, which always uses the position
# but if you use .loc, be prepared to have things go missing on your when you dropna
# of course, if your index is a bunch of strings, then that's totally fine...

# Exercise: Missing weather details

1. Create a series in which the index is days of the week, and the values are the projected high temperatures for where you live in the next 10 days.
2. Assign `NaN` to three of those values.
3. First, use `fillna` to replace those values with the mean and the median. Which seems to give closer/better values?
4. Next, use `dropna` to remove the `NaN` values. What happens now if you try to show a forecast? What advantages and disadvantages do you see?

In [7]:
s = Series([30, 30, 28, 28, 29, 30, 34, 26, 27, 28],
           index='Tue Wed Thu Fri Sat Sun Mon Tue Wed Thu'.split())
s

Tue    30
Wed    30
Thu    28
Fri    28
Sat    29
Sun    30
Mon    34
Tue    26
Wed    27
Thu    28
dtype: int64

In [8]:
s.loc[['Wed', 'Sat']] = np.nan
s

Tue    30.0
Wed     NaN
Thu    28.0
Fri    28.0
Sat     NaN
Sun    30.0
Mon    34.0
Tue    26.0
Wed     NaN
Thu    28.0
dtype: float64

In [9]:
s.mean()

29.142857142857142

In [10]:
s.median()

28.0

In [11]:
# fill in the values with mean 
s.fillna(s.mean())

Tue    30.000000
Wed    29.142857
Thu    28.000000
Fri    28.000000
Sat    29.142857
Sun    30.000000
Mon    34.000000
Tue    26.000000
Wed    29.142857
Thu    28.000000
dtype: float64

In [12]:
s.fillna(s.median())

Tue    30.0
Wed    28.0
Thu    28.0
Fri    28.0
Sat    28.0
Sun    30.0
Mon    34.0
Tue    26.0
Wed    28.0
Thu    28.0
dtype: float64

In [13]:
s.dropna()

Tue    30.0
Thu    28.0
Fri    28.0
Sun    30.0
Mon    34.0
Tue    26.0
Thu    28.0
dtype: float64

# Data frames and `NaN`

As a general rule, anything that we can do with a series, we can also do with a data frame. And when we do that, we get the result from applying the method to every single column.

In [14]:
np.random.seed(0)
df = DataFrame(np.random.randint(-500, 500, [3,4]),
               index=list('abc'),
               columns=list('wxyz'))
df

,w,x,y,z
a,184,59,129,-308
b,335,263,207,-141
c,-491,223,-223,254


In [15]:
df.loc['a', 'w'] = np.nan
df.loc['a', 'y'] = np.nan
df.loc['c', 'y'] = np.nan

df



,w,x,y,z
a,NaN,59,NaN,-308
b,335.0,263,207.0,-141
c,-491.0,223,NaN,254


In [16]:
# if I want to run fillna, I can with a scalar value
df.fillna(9999)

,w,x,y,z
a,9999.0,59,9999.0,-308
b,335.0,263,207.0,-141
c,-491.0,223,9999.0,254


In [17]:
# I can do better than that!
# I can say df.mean()

df.mean()

w    -78.000000
x    181.666667
y    207.000000
z    -65.000000
dtype: float64

In [18]:
# watch what happens when we now use df.fillna with the results of df.mean:

df.fillna(df.mean())

,w,x,y,z
a,-78.0,59,207.0,-308
b,335.0,263,207.0,-141
c,-491.0,223,207.0,254


In [19]:
# I can pass, if I want, a dict to fillna, whose keys are
# the column names. We can indicate what value(s) we want to 
# pass to each column.

df.fillna({'w':9999, 'y':df['y'].mean()})

,w,x,y,z
a,9999.0,59,207.0,-308
b,335.0,263,207.0,-141
c,-491.0,223,207.0,254


In [20]:
df

,w,x,y,z
a,NaN,59,NaN,-308
b,335.0,263,207.0,-141
c,-491.0,223,NaN,254


In [21]:
# what will happen when I use dropna?
# every row containing NaN will be removed
# (or really, will not be in the new data frame that's returned)

df.dropna()

,w,x,y,z
b,335.0,263,207.0,-141


In [22]:
# we can limit the degree to which nan is seen as a problem, and dropped. We can pass
# one or both of the following keyword arguments:

# 1. thresh, an integer indicating how many good values a row must have to be kept
# 2. subset, a list of column names (strings) that we want to look at when determining if we should drop the row
#   meaning: if a column is not in subset, then we don't look at it when making a decision

df.dropna(thresh=3)  # this means: If I have 3 values, I'm good!

,w,x,y,z
b,335.0,263,207.0,-141
c,-491.0,223,NaN,254


In [24]:
df

,w,x,y,z
a,NaN,59,NaN,-308
b,335.0,263,207.0,-141
c,-491.0,223,NaN,254


In [26]:
df.dropna(subset=['w', 'z'])

,w,x,y,z
b,335.0,263,207.0,-141
c,-491.0,223,NaN,254


# Exercise: Boston temperatures

1. Read the file at `../data/boston,ma.csv` and put into a CSV file. We only care about two columns, namely the min and max temperatures that were recorded during that period.
2. Put `NaN` values every 5 items in the maxtemp
3. Put `NaN` values every 3 items in the mintemp. (Note: For both of these, using `iloc` and a slice will really come in handy.)
4. How many rows remain (as a percentage) if we do a total `dropna`?
5. What if we only drop those rows where there is a `NaN` in the mintemp?
6. Replace the `NaN` values with the mean.

In [27]:
filename = '../data/boston,ma.csv'
!head $filename

date_time,"boston,ma_maxtempC","boston,ma_mintempC","boston,ma_totalSnow_cm","boston,ma_sunHour","boston,ma_uvIndex","boston,ma_uvIndex","boston,ma_moon_illumination","boston,ma_moonrise","boston,ma_moonset","boston,ma_sunrise","boston,ma_sunset","boston,ma_DewPointC","boston,ma_FeelsLikeC","boston,ma_HeatIndexC","boston,ma_WindChillC","boston,ma_WindGustKmph","boston,ma_cloudcover","boston,ma_humidity","boston,ma_precipMM","boston,ma_pressure","boston,ma_tempC","boston,ma_visibility","boston,ma_winddirDegree","boston,ma_windspeedKmph"
2018-12-11 00:00:00,1,-4,0.0,8.7,2,0,21,10:19 AM,08:12 PM,07:03 AM,04:11 PM,-7,-3,0,-3,10,0,57,0.0,1022,-3,10,339,8
2018-12-11 03:00:00,1,-4,0.0,8.7,2,0,21,10:19 AM,08:12 PM,07:03 AM,04:11 PM,-7,-1,1,-1,7,2,57,0.0,1023,-3,10,319,6
2018-12-11 06:00:00,1,-4,0.0,8.7,2,0,21,10:19 AM,08:12 PM,07:03 AM,04:11 PM,-9,-5,-3,-5,8,4,60,0.0,1023,-4,10,334,7
2018-12-11 09:00:00,1,-4,0.0,8.7,2,2,21,10:19 AM,08:12 PM,07:03 AM,04:11 PM,-9,1,1,1,3,6,49,0.0,1022,-1,10,334,

In [31]:
df = pd.read_csv(filename, usecols=[1, 2],
                 names=['maxtemp', 'mintemp'],
                header=0)

In [32]:
df.head()

,maxtemp,mintemp
0,1,-4
1,1,-4
2,1,-4
3,1,-4
4,1,-4


In [33]:
df.head()

,maxtemp,mintemp
0,1,-4
1,1,-4
2,1,-4
3,1,-4
4,1,-4


In [35]:
df.loc[::5, 'maxtemp'] = np.nan

In [36]:
df.loc[::3, 'mintemp'] = np.nan

In [37]:
df.head(20)

,maxtemp,mintemp
0,NaN,NaN
1,1.0,-4.0
2,1.0,-4.0
3,1.0,NaN
4,1.0,-4.0
5,NaN,-4.0
6,1.0,NaN
7,1.0,-4.0
8,2.0,-3.0
9,2.0,NaN


In [38]:
df.shape

(728, 2)

In [39]:
df.dropna().shape

(388, 2)

In [40]:
df.dropna(subset=['mintemp'])   # we only care if there's NaN in the mintemp

,maxtemp,mintemp
1,1.0,-4.0
2,1.0,-4.0
4,1.0,-4.0
5,NaN,-4.0
7,1.0,-4.0
...,...,...
721,8.0,2.0
722,8.0,2.0
724,8.0,2.0
725,NaN,2.0


In [42]:
# only drop rows if we have 2 nan values 
df.dropna(thresh=1).shape  # ignore 1-nan rows

(679, 2)

In [43]:
df.fillna(df.mean())

,maxtemp,mintemp
0,2.850515,-3.142268
1,1.000000,-4.000000
2,1.000000,-4.000000
3,1.000000,-3.142268
4,1.000000,-4.000000
...,...,...
723,8.000000,-3.142268
724,8.000000,2.000000
725,2.850515,2.000000
726,8.000000,-3.142268


# Interpolation

We now see that while it's easy to replace NaN with mean/median, it doesn't always make sense. Especially if we have data that's natrually on a linear curve, going up and then down without any wild swings, using the overall mean/median is going to make things stick out, not feel very natural.

If we, as humans, were to see a missing data point between 29 and 31, we would guess that it could/should be 30, and we would probably be right more often than wrong.

The `interpolate` method does exactly that: It replaces every `NaN` with the mean of the two values on either side. That way, if the data lends itself to such a linear curve, we can get values back that are roughly equivalent to what they should have been.

In [45]:
df['mintemp'].interpolate().head(20)

0     NaN
1    -4.0
2    -4.0
3    -4.0
4    -4.0
5    -4.0
6    -4.0
7    -4.0
8    -3.0
9    -3.0
10   -3.0
11   -3.0
12   -3.0
13   -3.0
14   -3.0
15   -4.0
16   -5.0
17   -5.0
18   -5.0
19   -5.0
Name: mintemp, dtype: float64

In [47]:
df.interpolate().head(20)

,maxtemp,mintemp
0,NaN,NaN
1,1.0,-4.0
2,1.0,-4.0
3,1.0,-4.0
4,1.0,-4.0
5,1.0,-4.0
6,1.0,-4.0
7,1.0,-4.0
8,2.0,-3.0
9,2.0,-3.0


In [49]:
!head -1000 ../../nyc-parking-violations-2020.csv > nyc-mini-parking-violations-2020.csv

In [50]:
filename = 'nyc-mini-parking-violations-2020.csv'

In [51]:
df = pd.read_csv(filename)

In [52]:
df.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1477633194,J58JKX,NJ,PAS,05/08/1972 12:00:00 AM,16,SDN,HONDA,P,8730,...,BK,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
1,1449715424,KRE6058,PA,PAS,08/29/1977 12:00:00 AM,98,SUBN,ME/BE,P,86530,...,BLK,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
2,1455779155,444326R,NJ,PAS,10/03/1988 12:00:00 AM,20,SDN,LEXUS,P,27030,...,BLACK,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
3,1458800908,F728330,OH,PAS,01/03/1990 12:00:00 AM,21,SDN,CHEVR,P,33030,...,NaN,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
4,1466038676,FMY9090,NY,PAS,02/14/1990 12:00:00 AM,21,SUBN,JEEP,S,45130,...,GREY,0.0,2015,-,0,NaN,NaN,NaN,NaN,NaN


In [54]:
df['Vehicle Color'].head(20)

0        BK
1       BLK
2     BLACK
3       NaN
4      GREY
5        GY
6        WH
7       BLK
8      GREY
9       RED
10     GREY
11     GRAY
12    BLACK
13    BLACK
14      TAN
15    SILVE
16       WH
17       BN
18       WH
19       WH
Name: Vehicle Color, dtype: object

In [55]:
df['Vehicle Color'].value_counts()

Vehicle Color
WH       137
GY       112
BK       110
BLACK     94
WHITE     86
GREY      56
BL        49
RED       35
BLUE      34
SILVE     28
BLK       26
GR        23
RD        20
BR        17
WHT       17
BROWN     15
GRAY      11
GRY       10
TAN        9
OTHER      9
YW         8
GREEN      7
W          7
BLU        5
TN         5
BW         4
GOLD       3
GL         3
GRN        2
YELLO      2
B          2
WT         2
BRN        2
GN         2
SL         1
ORANG      1
LTG        1
Y          1
PURPL      1
BN         1
MR         1
BLA        1
SIL        1
YELL       1
SLV        1
SILV       1
BLCK       1
SLVR       1
BRW        1
DKG        1
Name: count, dtype: int64

In [59]:
# we can use the replace method on a series (column) to replace one value with another
# we define this in a dict, which we pass to the method. The dict keys are the "from" 
# values and the dict values are the "to" values.

color_translation = {'BK':'BLACK', 'BLK':'BLACK', 'WH':'WHITE', 'GR':'GRAY', 'GREY':'GRAY',
                    'GY':'GRAY', 'BL':'BLUE', 'WHT':'WHITE'}

df['Vehicle Color'].replace(color_translation).value_counts()

Vehicle Color
WHITE    240
BLACK    230
GRAY     202
BLUE      83
RED       35
SILVE     28
RD        20
BR        17
BROWN     15
GRY       10
TAN        9
OTHER      9
YW         8
GREEN      7
W          7
TN         5
BLU        5
BW         4
GOLD       3
GL         3
GN         2
BRN        2
WT         2
B          2
GRN        2
YELLO      2
BLA        1
ORANG      1
LTG        1
BN         1
PURPL      1
MR         1
SLV        1
YELL       1
SL         1
SILV       1
BLCK       1
Y          1
SLVR       1
BRW        1
SIL        1
DKG        1
Name: count, dtype: int64

In [61]:
df.dropna().shape

(0, 43)

In [62]:
s = Series([10, 20, np.nan, 40, 50])
s

0    10.0
1    20.0
2     NaN
3    40.0
4    50.0
dtype: float64

In [63]:
s = Series('this is a text series'.split())
s

0      this
1        is
2         a
3      text
4    series
dtype: object

In [64]:
s.loc[2] = np.nan

In [65]:
s

0      this
1        is
2       NaN
3      text
4    series
dtype: object

# Nullable types

We want to have an integer series, but one that can contain NaN values. We want to have a string series, but one that can contain NaN values, as well. We don't want